In [ ]:
import os

cwd = os.getcwd()
os.chdir('/home/phbelang/abp/BBStudies/Executables/py_BB/xtrack/examples/twiss')


import numpy as np
from cpymad.madx import Madx
import xtrack as xt
import xpart as xp

mad_b2 = Madx(command_log="mad_collider.log")
mad_b2.call('../../test_data/hllhc15_noerrors_nobb/sequence.madx')
mad_b2.use(sequence='lhcb2')
twb2mad = mad_b2.twiss()
summb2mad = mad_b2.table.summ
survb2mad = mad_b2.survey().dframe()

mad_b4 = Madx(command_log="mad_collider.log")
mad_b4.call('../../test_data/hllhc15_noerrors_nobb/sequence_b4.madx')
mad_b4.use(sequence='lhcb2')
twb4mad = mad_b4.twiss()
summb4mad = mad_b4.table.summ
survb4mad = mad_b4.survey().dframe()

line_b4 = xt.Line.from_madx_sequence(mad_b4.sequence['lhcb2'],
                                     #deferred_expressions=True
                                     )
line_b4.particle_ref = xp.Particles(mass0=xp.PROTON_MASS_EV, p0c=7000e9)

tracker_b4 = xt.Tracker(line=line_b4)
twb4xt = tracker_b4.twiss()
survb4xt = tracker_b4.survey().to_pandas(index='name')


In [ ]:

import matplotlib.pyplot as plt
plt.close('all')


plt.figure()
plt.subplot(2,1,1)
plt.plot(survb2mad['s'], survb2mad['x'], 'r')
plt.plot(survb2mad.loc['ip2', 's'], survb2mad.loc['ip2', 'x'], 'o', color='r')
plt.plot(survb4mad['s'], survb4mad['x'], 'lightblue')
plt.plot(survb4mad.loc['ip2', 's'], survb4mad.loc['ip2', 'x'], 'o', color='lightblue')
plt.plot(survb4xt.s, survb4xt.X, '--', color='b')
plt.plot(survb4xt.loc['ip2', 's'], survb4xt.loc['ip2', 'X'], 'o', color='b')
plt.subplot(2,1,2)
plt.plot(survb2mad['s'], survb2mad['z'], 'r', lw=5)
plt.plot(survb4mad['s'], survb4mad['z'], 'lightblue')
plt.plot(survb4xt.s, survb4xt.Z, '--', color='b')
#plt.show()

from xtrack.survey_from_tracker import SurveyTable
def temp_mirror(self):
    new = SurveyTable()
    for kk, vv in self.items():
        new[kk] = vv

    for kk in new.keys():
        new[kk] = new[kk][::-1]

    new.s = new.s[0] - new.s

    return new

survb2xt = temp_mirror(tracker_b4.survey())

In [ ]:
2+2

In [ ]:
import os

cwd = os.getcwd()
os.chdir('../Executables/py_BB/xtrack/tests')

import pathlib
import numpy as np

import xtrack as xt
import xpart as xp
import xobjects as xo
import xfields as xf

from cpymad.madx import Madx

seq_name = 'lhcb1'
__file__ = '/home/phbelang/abp/BBStudies/Executables/py_BB/xtrack/tests/test_vs_madx.py' # REMOVE
test_data_folder = pathlib.Path(
        __file__).parent.joinpath('../test_data').absolute()

path = test_data_folder.joinpath('hllhc14_input_mad/')


mad_no_errors = Madx(command_log="mad_final.log")
mad_no_errors.call(str(test_data_folder.joinpath(
                               'hllhc15_noerrors_nobb/sequence.madx')))
mad_no_errors.use(sequence=seq_name)
mad_no_errors.globals['vrf400'] = 16
mad_no_errors.globals['lagrf400.b1'] = 0.5
mad_no_errors.twiss()


# start test_survey here:

mad = mad_no_errors
mad.survey()

srvmad = mad.table.survey.dframe()

line_full = xt.Line.from_madx_sequence(mad.sequence['lhcb1'])
line_full.particle_ref = xp.Particles(mass0=xp.PROTON_MASS_EV, q0=1,
                        gamma0=mad.sequence.lhcb1.beam.gamma)

tracker = xt.Tracker(line=line_full)

os.chdir(cwd)

In [ ]:
xsurvey   = tracker.survey()
srvXsuite = xsurvey.to_pandas(index='name')

display(srvmad)
display(srvXsuite)

In [ ]:
srvmad

In [ ]:
srvmad.loc[idx,'x']

In [ ]:
srvmad.loc[idx]

In [ ]:
xsurvey = tracker.survey().to_pandas(index='name')

for ridof in ['drift','#']:
    srvmad    =    srvmad[np.invert(srvmad.index.str.contains(ridof,regex=False))]

for coord in ['X','Z']:
    idx = srvmad[coord.lower()].index
    assert np.all(np.isclose(xsurvey.loc[idx,coord],srvmad.loc[idx,coord.lower()], atol=1e-9, rtol=0))


In [ ]:
%%timeit
tracker.survey()

In [ ]:

import pandas as pd

def get_w_from_angles(theta,phi,psi):
    '''W matrix, see MAD-X manual'''
    costhe = np.cos(theta)
    cosphi = np.cos(phi)
    cospsi = np.cos(psi)
    sinthe = np.sin(theta)
    sinphi = np.sin(phi)
    sinpsi = np.sin(psi)
    w       = np.zeros([3,3])
    w[0, 0] = + costhe * cospsi - sinthe * sinphi * sinpsi
    w[0, 1] = - costhe * sinpsi - sinthe * sinphi * cospsi
    w[0, 2] = sinthe * cosphi
    w[1, 0] = cosphi * sinpsi
    w[1, 1] = cosphi * cospsi
    w[1, 2] = sinphi
    w[2, 0] = - sinthe * cospsi - costhe * sinphi * sinpsi
    w[2, 1] = + sinthe * sinpsi - costhe * sinphi * cospsi
    w[2, 2] = costhe * cosphi
    return w


def get_angles_from_w(w):
    # w[0, 2]/w[2, 2] = (sinthe * cosphi)/(costhe * cosphi)
    # w[1, 0]/w[1, 1] = (cosphi * sinpsi)/(cosphi * cospsi)
    # w[1, 2]/w[1, 1] = (sinphi)/(cosphi * cospsi)

    theta = np.arctan2(w[0, 2], w[2, 2])
    psi   = np.arctan2(w[1, 0], w[1, 1])    
    phi   = np.arctan2(w[1, 2],w[1, 1]/np.cos(psi))

    return theta,phi,psi



def advance_bend(v,w,R,S):
    '''Advancing through elements, see MAD-X manual:
        v2 = w1*R + v1  | w2 = w1*S'''
    return np.dot(w,R)+v,np.dot(w,S)


def advance_drift(v,w,R):
    '''Advancing through elements, see MAD-X manual:
        v2 = w1*R + v1  | w2 = w1*S -> S is unity'''
    return np.dot(w,R)+v,w


def element_advance(v,w,length=0,angle=0,tilt=0):
    '''Computing the advance element-by-element. See MAD-X manual for generation of R and S'''
    if angle  == 0:
        R = np.array([0,0,length])
        return advance_drift(v,w,R)
    elif tilt == 0:
        # Relevant sine/cosine
        ca=np.cos(angle); sa=np.sin(angle)
        #------
        rho = length/angle
        R   = np.array([rho*(ca-1),0,rho*sa])
        S   = np.array([[ca , 0 ,-sa],
                        [0  , 1 ,  0],
                        [sa , 0 , ca]])
        return advance_bend(v,w,R,S)

    else:
        # Relevant sine/cosine
        ca=np.cos(angle); sa=np.sin(angle)
        ct=np.cos(tilt); st=np.sin(tilt)
        #------
        rho = length/angle
        R   = np.array([rho*(ca-1),0,rho*sa])
        S   = np.array([[ca , 0 ,-sa],
                        [0  , 1 ,  0],
                        [sa , 0 , ca]])

        # Orthogonal rotation matrix for tilt
        T   = np.array([[ct ,-st,  0],
                        [st , ct,  0],
                        [0  , 0 ,  1]])
        Tinv= np.array([[ct , st,  0],
                        [-st, ct,  0],
                        [0  , 0 ,  1]])

        return advance_bend(v,w,np.dot(T,R),np.dot(T,np.dot(S,Tinv)))
        



_is_thick = xt.line._is_thick #tmp

# TO ADD in xtrack.line
#========================================
def get_length_elements(self):
        lengths = []
        for ee in self.elements:
            if _is_thick(ee):
                lengths.append(ee.length)
            else:
                lengths.append(0.0)
        return lengths
#========================================


import types  # TMP                  
tracker.line.get_length_elements = types.MethodType(get_length_elements, tracker.line) # TMP


class SurveyTable(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self

    def to_pandas(self,index = None):
        import pandas as pd
        df = pd.DataFrame(self)
        if index is not None:
            df.set_index(index,inplace=True)
        return df


# SURVEY FROM TRACKER
#========================================

X0=0;Y0=0;Z0=0;theta0=0;phi0=0;psi0=0

survey_el_by_el = SurveyTable({ 'name'  : tracker.line.element_names + ('_end_point',),
                                's'     : np.array(tracker.line.get_s_elements() + [tracker.line.get_length()]),
                                'l'     : np.array(tracker.line.get_length_elements() + [0.0]),
                                'X'     : [X0],
                                'Y'     : [Y0],
                                'Z'     : [Z0],
                                'theta' : [theta0],
                                'phi'   : [phi0],
                                'psi'   : [psi0] })


v = np.array([X0,Y0,Z0])
w = get_w_from_angles(theta = theta0, phi = phi0, psi = psi0)
for ee,length,name in zip(tracker.line.elements[1:],survey_el_by_el['l'][1:-1],survey_el_by_el['name'][1:-1]):
    
    hxl,hyl = (ee.hxl,ee.hyl) if hasattr(ee ,'hxl') else (0,0)
    
    ##############
    # TODO Generalize for non-flat machines
    assert hyl == 0, f"Survey of machines with tilt not yet implemented, {name} has hyl={hyl} "
    
    angle = hxl
    tilt  = 0
    ##############

    # Advancing
    v,w = element_advance(v,w,length=length,angle=angle,tilt=tilt)

    # Unpacking results
    theta,phi,psi = get_angles_from_w(w)
    #----
    survey_el_by_el['X'].append(v[0])
    survey_el_by_el['Y'].append(v[1])
    survey_el_by_el['Z'].append(v[2])
    #----
    survey_el_by_el['theta'].append(theta)
    survey_el_by_el['phi'].append(phi)
    survey_el_by_el['psi'].append(psi)

# Repeating for endpoint
for _key in ['X','Y','Z','theta','phi','psi']:
    survey_el_by_el[_key].append(survey_el_by_el[_key][-1])



srvXsuite = survey_el_by_el.to_pandas(index='name')


In [ ]:
srvXsuite

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
plt.suptitle(f'LHC: {seq_name}')
plt.plot(srvmad['x']   ,srvmad['z']   ,'-',color='C0',alpha=0.9,label='MAD-X' ,lw=5)
plt.plot(srvXsuite['X'],srvXsuite['Z'],'-',color='C1',alpha=0.9,label='xsuite')
for _IP in ['ip1','ip5']:
    plt.plot(srvmad.loc[_IP,'x']   ,srvmad.loc[_IP,'z']   ,'o',color='C0',alpha=0.9,label=_IP,mfc='none')
    plt.plot(srvXsuite.loc[_IP,'X'],srvXsuite.loc[_IP,'Z'],'.',color='C1',alpha=0.9)
plt.legend()
plt.axis('equal')
plt.xlabel('x (m)')
plt.ylabel(r'z (m)')


plt.figure(figsize=(15,6))
plt.suptitle(f'LHC: {seq_name}')
plt.plot(srvmad['x']   ,srvmad['z']   ,'-o',color='C0',alpha=0.9,label='MAD-X' ,lw=1)
plt.plot(srvXsuite['X'],srvXsuite['Z'],'.',color='C1',alpha=0.9,label='xsuite')
for _IP in ['ip5']:
    plt.plot(srvmad.loc[_IP,'x']   ,srvmad.loc[_IP,'z']   ,'P',color='k',alpha=0.9,label=_IP,mfc='none')
    plt.plot(srvXsuite.loc[_IP,'X'],srvXsuite.loc[_IP,'Z'],'P',color='k',alpha=0.9)
plt.legend()
#plt.axis('square')
_IP = 'ip5'
plt.xlim([srvmad.loc[_IP,'x']-200,srvmad.loc[_IP,'x']+200])
plt.ylim([srvmad.loc[_IP,'z']-150e-3,srvmad.loc[_IP,'z']+150e-3])
plt.xlabel('x (m)')
plt.ylabel(r'z (m)')




In [ ]:
diffX = srvmad['x']-srvXsuite['X']
diffX.dropna()

In [ ]:


diffX = (srvmad['x']-srvXsuite['X']).dropna()
diffZ = (srvmad['z']-srvXsuite['Z']).dropna()

plt.figure()
plt.suptitle(f'LHC: {seq_name}')
plt.plot(srvXsuite.loc[diffX.index,'s'],np.abs(diffX),'.',alpha=0.5,label='diffX')
plt.plot(srvXsuite.loc[diffZ.index,'s'],np.abs(diffZ),'.',alpha=0.5,label='diffZ')

plt.legend()
plt.xlabel('s (m)')
plt.ylabel(r'diff')

In [ ]:


diffX = (srvmad['x']-srvXsuite['X']).dropna()
diffZ = (srvmad['z']-srvXsuite['Z']).dropna()

plt.figure()
plt.suptitle(f'LHC: {seq_name}')
plt.plot(srvXsuite.loc[diffX.index,'s'],np.abs(diffX/srvmad.loc[diffX.index,'x']),'.',alpha=0.5,label='diffX')
plt.plot(srvXsuite.loc[diffZ.index,'s'],np.abs(diffZ/srvmad.loc[diffZ.index,'z']),'.',alpha=0.5,label='diffZ')

plt.legend()
plt.ylim([1e-15,1])
plt.yscale('log')
plt.xlabel('s (m)')
plt.ylabel(r'Rel. diff')

In [ ]:
(srvmad['x']-srvXsuite['X']).loc['ip3']

In [ ]:
plt.figure()
plt.plot(srvmad.s,srvmad.theta)
plt.plot(srvXsuite.s,srvXsuite.theta)